<a href="https://colab.research.google.com/github/akhileshkumbhar05-ui/bioinformatics-course-project/blob/main/Bioinformatics_Course_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy requests tqdm

In [2]:
import os, json, tarfile
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

In [3]:
# ---- GDC configuration ----
PROJECT_ID = "TCGA-BRCA"

DATA_CATEGORY = "Transcriptome Profiling"
DATA_TYPE     = "Gene Expression Quantification"
WORKFLOW_TYPE = "STAR - Counts"

# We only want tumor vs normal
KEEP_SAMPLE_TYPES = ["Primary Tumor", "Solid Tissue Normal"]

# To prototype: set a small number (e.g. 60).
# Set to None to download ALL matching files (will be many GB).
MAX_FILES = 80

GDC_FILES_ENDPOINT = "https://api.gdc.cancer.gov/files"
GDC_DATA_ENDPOINT  = "https://api.gdc.cancer.gov/data"

DOWNLOAD_DIR = "gdc_tcga_brca_star"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

In [4]:
filters = {
    "op": "and",
    "content": [
        {
            "op": "in",
            "content": {
                "field": "cases.project.project_id",
                "value": [PROJECT_ID],
            },
        },
        {
            "op": "in",
            "content": {
                "field": "files.data_category",
                "value": [DATA_CATEGORY],
            },
        },
        {
            "op": "in",
            "content": {
                "field": "files.data_type",
                "value": [DATA_TYPE],
            },
        },
        {
            "op": "in",
            "content": {
                "field": "files.analysis.workflow_type",
                "value": [WORKFLOW_TYPE],
            },
        },
        {
            "op": "in",
            "content": {
                "field": "files.access",
                "value": ["open"],
            },
        },
    ],
}

# Ask GDC to also expand case / sample info so we can get sample_type, etc.
expand_fields = ",".join([
    "cases",
    "cases.samples",
    "cases.project",
    "cases.demographic",
])

params = {
    # IMPORTANT: filters are sent as a JSON **string**
    "filters": json.dumps(filters),
    "format": "JSON",
    "size": "10000",     # plenty large for BRCA RNA-seq
    "pretty": "true",
    "expand": expand_fields,
}

resp = requests.get(GDC_FILES_ENDPOINT, params=params)
resp.raise_for_status()
files_json = resp.json()
len(files_json["data"]["hits"])

1231

In [5]:
rows = []

for f in files_json["data"]["hits"]:
    file_id   = f.get("file_id")
    file_name = f.get("file_name")

    cases = f.get("cases", [])
    if not cases:
        continue
    case  = cases[0]
    case_id          = case.get("case_id")
    case_submitter   = case.get("submitter_id")
    project_id       = case.get("project", {}).get("project_id")
    demographic      = case.get("demographic", {}) or {}

    samples = case.get("samples", [])
    # Take first sample attached to this file (RNA-seq quant is 1:1 with sample)
    sample = samples[0] if samples else {}
    sample_submitter = sample.get("submitter_id")    # e.g. TCGA-XX-XXXX-01A
    sample_type      = sample.get("sample_type")     # "Primary Tumor", "Solid Tissue Normal", etc.

    rows.append({
        "file_id": file_id,
        "file_name": file_name,
        "case_id": case_id,
        "case_submitter_id": case_submitter,
        "sample_submitter_id": sample_submitter,
        "sample_type": sample_type,
        "project_id": project_id,
        "gender": demographic.get("gender"),
        "age_at_diagnosis": demographic.get("age_at_diagnosis"),
    })

meta = pd.DataFrame(rows)
print("Total STAR-Counts files:", meta.shape)

# Keep only Primary Tumor and Solid Tissue Normal
meta = meta[meta["sample_type"].isin(KEEP_SAMPLE_TYPES)].copy()
meta = meta.sort_values(["sample_type", "sample_submitter_id"]).reset_index(drop=True)
print("After filtering to Tumor/Normal:", meta.shape)

# Optional: subsample for testing
if MAX_FILES is not None and len(meta) > MAX_FILES:
    meta = meta.sample(MAX_FILES, random_state=0).reset_index(drop=True)
    print("Using subset for prototype:", meta.shape)

meta.head()


Total STAR-Counts files: (1231, 9)
After filtering to Tumor/Normal: (1224, 9)
Using subset for prototype: (80, 9)


,file_id,file_name,case_id,case_submitter_id,sample_submitter_id,sample_type,project_id,gender,age_at_diagnosis
0,c003fb3b-179c-49b0-8d9a-656c4dc39901,5fc65e17-1d90-411d-b01d-f6aaa72730f9.rna_seq.a...,37242f5a-25ae-4b1f-9ce6-09ce1dc92539,TCGA-E2-A15L,TCGA-E2-A15L-01A,Primary Tumor,TCGA-BRCA,female,None
1,a1a6d1dd-a1c6-4c5c-b54d-42de80680bc3,dc857517-6b3d-4008-85a4-a2b35efb1e7a.rna_seq.a...,86c6f993-327f-4525-9983-29c55625593a,TCGA-5L-AAT0,TCGA-5L-AAT0-01A,Primary Tumor,TCGA-BRCA,female,None
2,481c702a-58ee-4fab-9559-434a45615aa5,dc3a76a7-bac2-43c0-90e2-e251e459c659.rna_seq.a...,a8b1f6e7-2bcf-460d-b1c6-1792a9801119,TCGA-E9-A1NF,TCGA-E9-A1NF-01A,Primary Tumor,TCGA-BRCA,female,None
3,79bfddcc-1d7b-43a7-9221-8f230eca31d8,0a9e33db-2527-4cc3-8669-a7c10fed7a7f.rna_seq.a...,09765b0a-94f6-47d2-af56-93368084ac3a,TCGA-A7-A0CD,TCGA-A7-A0CD-01A,Primary Tumor,TCGA-BRCA,female,None
4,d0dcec39-c407-4593-98e3-133714cf2f7b,f18a5a92-2b2e-4834-8610-df98760db5aa.rna_seq.a...,c0b7b798-3383-4a45-a455-9eca5810739e,TCGA-EW-A3E8,TCGA-EW-A3E8-01B,Primary Tumor,TCGA-BRCA,female,None


In [6]:
meta.to_csv("tcga_brca_star_metadata.csv", index=False)
print("Saved metadata to tcga_brca_star_metadata.csv")

Saved metadata to tcga_brca_star_metadata.csv


In [7]:
def download_chunk(file_ids, chunk_idx):
    """
    Download a chunk of files from GDC /data and extract into DOWNLOAD_DIR.
    """
    if not file_ids:
        return

    payload = {"ids": file_ids}
    headers = {"Content-Type": "application/json"}

    r = requests.post(GDC_DATA_ENDPOINT,
                      data=json.dumps(payload),
                      headers=headers,
                      stream=True)
    r.raise_for_status()

    tar_path = os.path.join(DOWNLOAD_DIR, f"gdc_chunk_{chunk_idx:03d}.tar.gz")
    with open(tar_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)

    # Extract
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(DOWNLOAD_DIR)
    os.remove(tar_path)

# ---- Run the downloads ----
file_ids = meta["file_id"].tolist()
print("Files to download:", len(file_ids))

CHUNK_SIZE = 20

for i in tqdm(range(0, len(file_ids), CHUNK_SIZE)):
    chunk_ids = file_ids[i:i + CHUNK_SIZE]
    download_chunk(chunk_ids, i // CHUNK_SIZE)


Files to download: 80


  0%|          | 0/4 [00:00<?, ?it/s]/tmp/ipython-input-2452505344.py:25: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(DOWNLOAD_DIR)
100%|██████████| 4/4 [01:24<00:00, 21.22s/it]


In [8]:
import os
import pandas as pd

def read_star_counts_file(path: str) -> pd.Series:
    """
    Read one STAR counts TSV and return a Series:
      index = gene_id (no version suffix)
      values = unstranded counts
    Tries several layouts and is defensive.
    """
    # First try: normal headered TSV
    try:
        df = pd.read_csv(
            path,
            sep="\t",
            comment="#",
            engine="python"
        )
    except Exception as e:
        print(f"[WARN] {os.path.basename(path)}: header read failed ({e}), trying header=None")
        df = pd.read_csv(
            path,
            sep="\t",
            comment="#",
            engine="python",
            header=None
        )

    if {"gene_id", "unstranded"}.issubset(df.columns):
        # GDC "gene_counts.tsv" layout
        sub = df[["gene_id", "unstranded"]].copy()
        sub = sub[~sub["gene_id"].astype(str).str.startswith("N_")]
    else:
        # Fallback: assume first two columns are gene + count (ReadsPerGene style)
        # If headerless: first four rows are summary -> skip
        if df.shape[1] < 2:
            raise ValueError(f"Unexpected shape {df.shape} in {os.path.basename(path)}")

        if isinstance(df.columns[0], int):
            # likely header=None layout, skip first 4 rows
            df2 = df.iloc[4:, [0, 1]].copy()
        else:
            df2 = df.iloc[:, :2].copy()

        df2.columns = ["gene_id", "unstranded"]
        sub = df2

    # Remove Ensembl version suffix
    sub["gene_id"] = sub["gene_id"].astype(str).str.split(".").str[0]

    # Drop duplicates if any
    sub = sub.drop_duplicates("gene_id")

    # Return as Series
    s = sub.set_index("gene_id")["unstranded"]
    s.name = os.path.basename(path)
    return s


In [9]:
def build_counts_matrix(meta: pd.DataFrame, data_dir: str) -> pd.DataFrame:
    """
    Build gene x sample matrix from STAR counts TSVs in data_dir.

    - Uses sample_submitter_id as sample name
    - Only loads one file at a time into memory
    """
    # map filename -> sample_submitter_id
    file_to_sample = dict(zip(meta["file_name"], meta["sample_submitter_id"]))

    # index all TSV paths once
    path_by_name = {}
    for root, _, files in os.walk(data_dir):
        for fname in files:
            if fname.endswith(".tsv") and fname in file_to_sample:
                path_by_name[fname] = os.path.join(root, fname)

    print("TSV files that match metadata:", len(path_by_name))

    series_list = []
    sample_names = []
    skipped = []

    for i, (fname, sample_name) in enumerate(file_to_sample.items(), start=1):
        path = path_by_name.get(fname)
        if path is None:
            skipped.append((fname, "not found on disk"))
            continue

        try:
            s = read_star_counts_file(path)
        except Exception as e:
            print(f"[WARN] skipping {fname} ({e})")
            skipped.append((fname, str(e)))
            continue

        s.name = sample_name
        series_list.append(s)
        sample_names.append(sample_name)

        if i % 10 == 0:
            print(f"Read {i} files")

    if not series_list:
        raise RuntimeError("No files successfully read; cannot build matrix.")

    # Single concat at the end -> much lower RAM use
    mat = pd.concat(series_list, axis=1)
    mat.columns = sample_names

    # Optional: drop pseudo genes on PAR_Y
    mask = ~mat.index.to_series().str.contains("PAR_Y", na=False)
    mat = mat.loc[mask].copy()

    # Fill missing counts with 0 and cast to int
    mat = mat.fillna(0).astype(int)

    print("Final counts matrix shape (genes x samples):", mat.shape)
    print("Skipped files:", len(skipped))
    return mat


In [10]:
# For testing – keep first 20 samples
meta_sub = meta.head(20).copy()

rna_counts = build_counts_matrix(meta_sub, DOWNLOAD_DIR)
rna_counts.shape
rna_counts.head()


TSV files that match metadata: 20
Read 10 files
Read 20 files
Final counts matrix shape (genes x samples): (60616, 20)
Skipped files: 0


,TCGA-E2-A15L-01A,TCGA-5L-AAT0-01A,TCGA-E9-A1NF-01A,TCGA-A7-A0CD-01A,TCGA-EW-A3E8-01B,TCGA-E9-A22G-01A,TCGA-OL-A66J-01A,TCGA-EW-A1PD-01A,TCGA-A2-A0EM-01A,TCGA-B6-A401-01A,TCGA-BH-A1FE-11B,TCGA-C8-A133-01A,TCGA-A1-A0SN-01A,TCGA-A2-A0EP-01A,TCGA-E2-A1IL-01A,TCGA-A7-A3J0-01A,TCGA-AC-A2BK-01A,TCGA-E2-A155-01A,TCGA-BH-A0B8-11A,TCGA-E2-A10B-01A
gene_id,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,1061,2878,950,5729,2542,6767,1277,5617,6723,3380,7256,1581,837,2714,2703,301,8543,3148,3485,3119
ENSG00000000005,9,5,2,36,345,4,504,148,13,34,476,7,1,253,154,9,0,12,728,12
ENSG00000000419,2075,918,1034,2029,2300,2335,1853,1994,1503,2350,2500,1336,5465,1940,1154,1949,3453,2303,1387,3025
ENSG00000000457,2592,679,734,1590,1565,2772,1019,1335,2111,1665,2300,622,1871,2960,1701,2046,3528,1860,749,2211
ENSG00000000460,758,242,252,763,645,2037,276,678,998,758,672,112,844,552,479,508,1948,637,184,1254


In [11]:
# Now try with all samples in meta
rna_counts = build_counts_matrix(meta, DOWNLOAD_DIR)
rna_counts.shape


TSV files that match metadata: 80
Read 10 files
Read 20 files
Read 30 files
Read 40 files
Read 50 files
Read 60 files
Read 70 files
Read 80 files
Final counts matrix shape (genes x samples): (60616, 80)
Skipped files: 0


(60616, 80)

In [12]:
rna_counts.to_csv("tcga_brca_star_counts_matrix.csv")
meta.to_csv("tcga_brca_star_metadata.csv", index=False)
